In [1]:
import numpy as np
import pandas as pd
from sklearn import metrics, manifold
from tqdm import tqdm
import gensim.downloader as gensim_api
import transformers
import os

In [2]:
df=pd.read_csv('output.csv')
print(df)

                              je racist if other race
0   whenwwourdebit card decliiig athe abortion ann...
1                                       ken nein g y 
2                             si the wort you re loo 
3    ainerican teenagegirl visiting the children o...
4                                               taos 
5   ijustioundoutmexiean babies come out justilike...
6   when the schoolishooter accidently shoots thea...
7                                lsiiiniongutimnss s 
8              on oa oo wa that faggot am lae rat at 
9                                         ugly becta 
10                                 nigger up fc el ae
11                                       d it faggot 
12  oh you re a nigger ie ye a please telumehow mu...


In [3]:
nlp = gensim_api.load("glove-wiki-gigaword-300")

In [4]:
nlp.most_similar(["nigger"], topn=3)

[('slur', 0.5023007392883301),
 ('niggers', 0.5003144145011902),
 ('faggot', 0.4708784222602844)]

In [5]:
def get_similar_words(lst_words, top, nlp):
    lst_out = lst_words
    for tupla in nlp.most_similar(lst_words, topn=top):
        lst_out.append(tupla[0])
    return list(set(lst_out))

In [6]:
def similarWords(words, top, nlp):
    out = words
    for tupl in nlp.most_similar(words, topn=top):
        out.append(tupl[0])
    return list(set(out))

In [7]:
clusters={}
clusters['Hate']=get_similar_words(['nigger'], top=30, nlp=nlp)

In [8]:
tokenizer = transformers.DistilBertTokenizer.from_pretrained('al', do_lower_case=True)
nlp = transformers.TFDistilBertModel.from_pretrained('al')

HTTPError: 404 Client Error: Not Found for url: https://huggingface.co/api/models/al

In [ ]:
def utils_bert_embedding(txt, tokenizer, nlp):
    '''
    Word embedding with Bert (equivalent to nlp["word"]).
    :parameter
        :param txt: string 
        :param tokenizer: transformers tokenizer
        :param nlp: transformers bert
    :return
        tensor sentences x words x vector (1x3x768) 
    '''
    # tokenize sentence to tokens (integers)
    idx = tokenizer.encode(txt)
    # convert to array of shape (1, num_words+2) - EOS and CLS added
    idx = np.array(idx)[None,:]
    # generate embeddings for each token - output is a tuple
    embedding = nlp(idx)
    # select first member of the tuple, remove first dimension which is 1 to get (num_words,embedding size 712)
    # exclude CLS and EOS tokens
    X = np.array(embedding[0][0][1:-1])
    return X

In [ ]:
%%time
## create list of news vector
lst_mean_vecs = [utils_bert_embedding(txt, tokenizer, nlp).mean(0) for txt in tqdm(df['tweet'])]

In [ ]:
X = np.array(lst_mean_vecs)
X.shape

In [ ]:
np.savetxt("mean_vec1.csv", X, delimiter="/")

In [ ]:
print(X)

In [ ]:
from numpy import genfromtxt
my_data = genfromtxt('mean_vec1.csv', delimiter='/')
print(my_data)

In [ ]:
dic_y = {k:utils_bert_embedding(v, tokenizer, nlp).mean(0) for k,v in tqdm(clusters.items())}

In [ ]:
print(dic_y.keys())

In [ ]:
def fix_NAN_inf_values(x):
    '''Replace NaN with zero and infinity with large finite numbers'''
    if len(np.where(np.isnan(X))[0])>0 or len(np.where(np.isnan(X))[1])>0:
        return np.nan_to_num(X)

In [ ]:
similarities = np.array(
            [metrics.pairwise.cosine_similarity(X, [dic_y['Hate']]).T.tolist()[0]]
            ).T

In [ ]:
print(similarities)

In [ ]:
dtf=pd.read_csv('full_data/hate_speech_mlma/hate_speech_mlma/fr_dataset.csv')


In [ ]:
print(dtf['sentiment'])

In [ ]:
z=np.array(dtf['sentiment'])
print(z[21])

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
plt.plot(similarities)

In [ ]:
y=[]
for similarity in similarities:
    y.append(similarity[0])

x=np.arange(df.shape[0])
plt.bar(x,height=y)
plt.show()

In [ ]:
plt.plot(z[:200])

In [ ]:
# relevant=(z == 0).sum()+(z == 1).sum()
# print(relevant)

In [ ]:
retrieved=(similarities>0.49).sum()
print(retrieved)


In [ ]:
# tp=0
# for i in range (0,4000):
#     if(similarities[i]>0.50 and z[i]<2):
#         tp=tp+1
# print(tp)

In [ ]:
# precision=tp/retrieved
# print("Precision is",precision)

In [ ]:
# fn=0
# for i in range (0,4000):
#     if(similarities[i]<0.50 and z[i]<2):
#         fn=fn+1
# recall=tp/(fn+tp)
# print("Recall is",recall)

In [ ]:
# fp=retrieved-tp
# tn=0
# for i in range (0,4000):
#     if(similarities[i]<0.50 and z[i]==2):
#         tn=tn+1
# accuracy=(tp+tn)/(tp+tn+fn+fp)
# print("Accuracy is",accuracy)